### Note

You can find the google colab version of this notebook [here](https://drive.google.com/file/d/18jIVEbE_1QbwTbHdMqj0AVqguf2Vrtrc/view?usp=sharing).

In [18]:
# !pip install polars polars-h3 matplotlib folium


In [19]:
import polars as pl
import polars_h3 as plh3

!mkdir -p sample-data
!wget https://sergey-filimonov.nyc3.cdn.digitaloceanspaces.com/polars-h3/sample-data/sample-telematics-data.parquet -O sample-data/sample-telematics-data.parquet


--2025-07-17 22:31:05--  https://sergey-filimonov.nyc3.cdn.digitaloceanspaces.com/polars-h3/sample-data/sample-telematics-data.parquet
Resolving sergey-filimonov.nyc3.cdn.digitaloceanspaces.com (sergey-filimonov.nyc3.cdn.digitaloceanspaces.com)... 172.64.145.29, 104.18.42.227
Connecting to sergey-filimonov.nyc3.cdn.digitaloceanspaces.com (sergey-filimonov.nyc3.cdn.digitaloceanspaces.com)|172.64.145.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67718 (66K) [text/plain]
Saving to: ‘sample-data/sample-telematics-data.parquet’

sample-data/sample- 100%[===================>]  66.13K  --.-KB/s    in 0.002s  

2025-07-17 22:31:05 (33.1 MB/s) - ‘sample-data/sample-telematics-data.parquet’ saved [67718/67718]



# Advanced Notebooks: Working with Telematics Data 🚗

When working with telematics data, you'll often see it formatted with a GPS coordinate along of a timestamp. This can be challenging to analyze - we'll explore a case study on how h3 can be used to help.

### Case Study: Analyzing Delivery Route Efficiency 🚚
Logistics and delivery companies often want to know where their drivers are spending the most time. By mapping telematics data to H3 hexes, they can identify slow-moving segments of their delivery routes. If certain hex cells consistently lead to delays, these areas can be flagged for route re-planning. 

Let's imagine we're tasked with finding how long our users are spending in a given area, given the following data that we've collected from their vehicles. 

In [20]:
# Real world telematics data tends to be messy, so we're going to take a small fraction of the available data to work with.

df = pl.read_parquet(
    "./sample-data/sample-telematics-data.parquet",
).sample(fraction=0.15, seed=42)

df.head()


lat,long,timestamp,user_id
f64,f64,datetime[μs],str
39.984222,-105.001856,2024-12-16 03:19:57.328863,"""sergey"""
39.842133,-104.678345,2024-12-16 03:46:01.814648,"""sergey"""
39.837963,-104.687351,2024-12-16 03:41:59.725038,"""sergey"""
39.698351,-105.717406,2024-12-16 07:23:38.309796,"""sergey"""
39.745424,-105.436932,2024-12-16 07:00:56.919874,"""sergey"""


## Let's try visualizing the data and see what we're working with. 

In [21]:
# Make sure you have volume and mapplotlib installed to use the graphing functions
# !uv pip install folium matplotlib

In [22]:
# IMPORTANT: Notice all the gaps in the data -
# There's no particular reason to use hexes to visualize the points, but it's a good exercise.

plotting_df = df.with_columns(
    [plh3.latlng_to_cell(pl.col("lat"), pl.col("long"), 9).alias("hex9")]
)

plh3.graphing.plot_hex_outlines(plotting_df, hex_id_col="hex9")


### 🌐 Attaching trip ids to the data
One of the first challenges when getting an export of telematics data can actually be figuring out how to attach trip ids to the data.  

Trip IDs are essential because they segment telematics data into distinct trips, making analysis possible. Without them, the data is an unstructured stream of GPS points, and you can't identify where trips start or end, measure time spent in specific areas, or tie data to real-world events like geofencing alerts or delivery stops. 

In [23]:
FIFTEEN_MINUTES = 15 * 60
hex_col = "hex9"


def attach_trip_ids(df: pl.DataFrame, partition_columns: list[str]) -> pl.DataFrame:
    REQUIRED_COLUMNS = partition_columns + ["timestamp", "lat", "long"]

    return (
        df.unique()
        .select(REQUIRED_COLUMNS)
        .sort(partition_columns + ["timestamp"])
        .with_columns(
            [
                (pl.col("timestamp").diff().dt.total_milliseconds() / pl.lit(1000))
                .over(partition_columns)
                .alias("elapsed_seconds"),
                pl.col("lat").round(5),  # 1 meter precision
                pl.col("long").round(5),  # 1 meter precision
            ]
        )
        .with_columns(
            (
                (pl.col("elapsed_seconds") > FIFTEEN_MINUTES)
                .fill_null(False)
                .cast(pl.Boolean)
                .cum_sum()
                .alias("user_trip_id")
            ).over(partition_columns)  # Creating trip ids within each group
        )
        .with_columns(
            pl.concat_str(
                [pl.col(col).cast(pl.Utf8) for col in partition_columns]
                + [pl.col("user_trip_id").cast(pl.Utf8)],
                separator="-",
            ).alias("trip_id")  # global trip id
        )
        .drop(["user_trip_id"])
        .sort(partition_columns + ["timestamp"])
        .with_columns(
            [
                plh3.latlng_to_cell(
                    pl.col("lat"), pl.col("long"), 9, return_dtype=pl.Utf8
                ).alias(hex_col)
            ]
        )
    )


trips = attach_trip_ids(df, ["user_id"])
display(trips.head())

# Looks like we're working with 2 distinct trips.
display(trips["trip_id"].value_counts())

user_id,timestamp,lat,long,elapsed_seconds,trip_id,hex9
str,datetime[μs],f64,f64,f64,str,str
"""sergey""",2024-12-16 02:51:11.963989,40.0189,-105.27893,null,"""sergey-0""","""892681a306fffff"""
"""sergey""",2024-12-16 02:51:38.171209,40.01919,-105.27903,26.207,"""sergey-0""","""892681a306fffff"""
"""sergey""",2024-12-16 02:52:58.591218,40.01686,-105.27964,80.42,"""sergey-0""","""892681a33d3ffff"""
"""sergey""",2024-12-16 02:53:45.142662,40.01572,-105.27923,46.551,"""sergey-0""","""892681a33d7ffff"""
"""sergey""",2024-12-16 02:53:52.901236,40.01517,-105.27904,7.758,"""sergey-0""","""892681a33d7ffff"""


trip_id,count
str,u32
"""sergey-0""",188
"""sergey-1""",422


### We now have a better idea of how long the user spent in each hexagon. 

In [24]:
plh3.graphing.plot_hex_fills(
    trips.drop_nulls(), hex_id_col=hex_col, metric_col="elapsed_seconds"
)


## 🧭 Finding the hexes that were traveled across. 

In [25]:
prev_hex_col = f"prev_{hex_col}"

trips_with_hex = (
    trips.with_columns(pl.col(hex_col).shift(1).over("trip_id").alias(prev_hex_col))
    .filter(pl.col("elapsed_seconds") <= FIFTEEN_MINUTES)
    .rename({"timestamp": "end", hex_col: "end_hex", prev_hex_col: "start_hex"})
    .with_columns(
        [
            (pl.col("end") - pl.duration(seconds=pl.col("elapsed_seconds"))).alias(
                "start"
            ),
            plh3.grid_path_cells(pl.col("start_hex"), pl.col("end_hex")).alias("path"),
        ]
    )
    .drop(["start_hex", "end_hex", "lat", "long"])
    .sort(["trip_id", "start"])
    .select(
        [
            "user_id",
            "trip_id",
            "start",
            "end",
            "elapsed_seconds",
            "path",
        ]
    )
)

# we now hav a dataframe with hex
display(trips_with_hex.head())

user_id,trip_id,start,end,elapsed_seconds,path
str,str,datetime[μs],datetime[μs],f64,list[str]
"""sergey""","""sergey-0""",2024-12-16 02:51:12.171209,2024-12-16 02:51:38.171209,26.207,"[""892681a306fffff""]"
"""sergey""","""sergey-0""",2024-12-16 02:51:38.591218,2024-12-16 02:52:58.591218,80.42,"[""892681a306fffff"", ""892681a33d3ffff""]"
"""sergey""","""sergey-0""",2024-12-16 02:52:59.142662,2024-12-16 02:53:45.142662,46.551,"[""892681a33d3ffff"", ""892681a33d7ffff""]"
"""sergey""","""sergey-0""",2024-12-16 02:53:45.901236,2024-12-16 02:53:52.901236,7.758,"[""892681a33d7ffff""]"
"""sergey""","""sergey-0""",2024-12-16 02:53:53.297671,2024-12-16 02:54:12.297671,19.396,"[""892681a33d7ffff""]"


## ⏰ Figuring out time spent in each individual hexagon

Currently, our hex data is a list. The next part of this is figuring out exactly how much time the user spent in each hexagon.  This is a non-trivial calculation because the user may be moving at different speeds at different points within the trip. 

In [26]:
# This is not exactly needed, but it is more accurate to distribute time across hexas evenly.

trips_with_time_spent_in_hex = (
    trips_with_hex.sort(["trip_id", "start"])
    .with_columns(
        [
            # Get the length of the path
            pl.col("path").list.len().alias("path_length"),
        ]
    )
    .with_columns(
        [
            # Create a range of indices for the path
            pl.int_ranges(pl.lit(0), "path_length").alias("path_index"),
            # Calculate the time that should be allocated to each hexagon
            (pl.col("elapsed_seconds") / pl.col("path_length")).alias(
                "elapsed_seconds_per_hex"
            ),
        ]
    )
    .select(
        [
            "trip_id",
            "path",
            "end",
            "start",
            "path_index",
            "elapsed_seconds_per_hex",
        ]
    )
    .explode(["path", "path_index"])
    .with_columns(
        # We need to carefully allocate time to each hex.
        [
            # Calculate the end time for each hexagon
            (
                pl.col("start")
                + pl.duration(
                    seconds=(
                        pl.col("elapsed_seconds_per_hex") * (pl.col("path_index") + 1)
                    )
                )
            ).alias("end"),
            # Calculate the start time for each hexagon
            (
                pl.col("start")
                + pl.duration(
                    seconds=(pl.col("elapsed_seconds_per_hex") * pl.col("path_index"))
                )
            ).alias("start"),
        ]
    )
    # cleaning up the data
    .rename(
        {
            "path": hex_col,
            "elapsed_seconds_per_hex": "elapsed_seconds",
        }
    )
    .drop(["path_index"])
)


trips_with_time_spent_in_hex


trip_id,hex9,end,start,elapsed_seconds
str,str,datetime[μs],datetime[μs],f64
"""sergey-0""","""892681a306fffff""",2024-12-16 02:51:38.171209,2024-12-16 02:51:12.171209,26.207
"""sergey-0""","""892681a306fffff""",2024-12-16 02:52:18.591218,2024-12-16 02:51:38.591218,40.21
"""sergey-0""","""892681a33d3ffff""",2024-12-16 02:52:58.591218,2024-12-16 02:52:18.591218,40.21
"""sergey-0""","""892681a33d3ffff""",2024-12-16 02:53:22.142662,2024-12-16 02:52:59.142662,23.2755
"""sergey-0""","""892681a33d7ffff""",2024-12-16 02:53:45.142662,2024-12-16 02:53:22.142662,23.2755
…,…,…,…,…
"""sergey-1""","""89268162117ffff""",2024-12-16 07:52:59.207001,2024-12-16 07:52:33.207001,26.245
"""sergey-1""","""89268162117ffff""",2024-12-16 07:53:03.978851,2024-12-16 07:52:59.978851,4.771
"""sergey-1""","""89268162117ffff""",2024-12-16 07:53:08.750701,2024-12-16 07:53:04.750701,4.771


## 🎁 Cleaning up the final data 

In [27]:
# There's a lot of repeated hexes, so let's go ahead and aggregate that.

final_df = trips_with_time_spent_in_hex.group_by(
    ["trip_id", hex_col],
    maintain_order=True,
).agg(
    [
        pl.col("end").max(),
        pl.col("start").min(),
        pl.col("elapsed_seconds").sum(),
    ],
)

display(final_df.head())

trip_id,hex9,end,start,elapsed_seconds
str,str,datetime[μs],datetime[μs],f64
"""sergey-0""","""892681a306fffff""",2024-12-16 02:52:18.591218,2024-12-16 02:51:12.171209,66.417
"""sergey-0""","""892681a33d3ffff""",2024-12-16 02:53:22.142662,2024-12-16 02:52:18.591218,63.4855
"""sergey-0""","""892681a33d7ffff""",2024-12-16 02:54:33.969828,2024-12-16 02:53:22.142662,71.7655
"""sergey-0""","""892681a33c7ffff""",2024-12-16 02:55:14.762698,2024-12-16 02:54:33.969828,40.732
"""sergey-0""","""892681a331bffff""",2024-12-16 02:55:45.038420,2024-12-16 02:55:14.762698,31.0335


## Let's try visualizing 

In [29]:
# Notice how there's no gaps and we can see the time spent in each hexagon.
hex_map = plh3.graphing.plot_hex_fills(
    trips_with_time_spent_in_hex, hex_id_col=hex_col, metric_col="elapsed_seconds"
)
display(hex_map)